<a href="https://colab.research.google.com/github/vedika0806/images/blob/main/KlingAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# 🟢 Step 1: Install deps
!pip -q install requests

import requests, time, json, jwt
from IPython.display import HTML

ak = "ArENHtabMBytKhMGMGG3YYbTenyygDRP" # fill access key
sk = "AED9pyFkDbFekC99Rr4hKFQyy3CJbEne" # fill secret key

def encode_jwt_token(ak, sk):
    headers = {
        "alg": "HS256",
        "typ": "JWT"
    }
    payload = {
        "iss": ak,
        "exp": int(time.time()) + 1800, # The valid time, in this example, represents the current time+1800s(30min)
        "nbf": int(time.time()) - 5 # The time when it starts to take effect, in this example, represents the current time minus 5s
    }
    token = jwt.encode(payload, sk, headers=headers)
    return token

# 🟢 Step 2: Set your API key
API_KEY = encode_jwt_token(ak, sk)  # replace with your real KlingAI key
API_URL = "https://api.klingai.com/v1/videos/image2video"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# 🟢 Step 3: Define your request payload
payload = {
    "model_name": "kling-v1-6",
    "mode": "pro",   #try "fast" first, then "pro" if your plan allows
    "duration": 10,
    "image": "https://raw.githubusercontent.com/vedika0806/images/main/image%20(1).jpeg",  # 👈 direct image
    "prompt": "The girl smiled and waved while drinking coffee",
    "cfg_scale": 0.5,
}
    #"static_mask": "https://h2.inkwai.com/bs2/upload-ylab-stunt/ai_portal/1732888177/cOLNrShrSO/static_mask.png",
    #"dynamic_masks": [
      #{
        #"mask": "https://h2.inkwai.com/bs2/upload-ylab-stunt/ai_portal/1732888130/WU8spl23dA/dynamic_mask_1.png",
        #"trajectories": [
          #{"x":279,"y":219},{"x":417,"y":65}
       # ]
      #}
    #]
#}

# 🟢 Step 4: Submit the job
resp = requests.post(API_URL, headers=headers, json=payload)
print(resp)
job = resp.json()
print("Submit response:", json.dumps(job, indent=2))

job_id = job.get("data", {}).get("task_id") or job.get("data", {}).get("id")
assert job_id, f"No job id found! Full response:\n{json.dumps(job, indent=2)}"
print("Job ID:", job_id)

# 🟢 Step 5: Poll until finished

status_url = f"https://api.klingai.com/v1/videos/image2video/{job_id}"
video_url = None

for i in range(180):  # poll up to ~5 minutes
    r = requests.get(status_url, headers=headers).json()
    print(f"[{i}] status:", json.dumps(r, indent=2))

    data = r.get("data", {})
    state = data.get("task_status")
    if state == "succeed":
        result = data.get("result", {})
        video_url = result.get("video_url") or data.get("video_url")
        break
    elif state == "failed":
        raise RuntimeError(f"❌ Task failed: {json.dumps(r, indent=2)}")

    time.sleep(5)

assert video_url, f"No video_url found! Final response:\n{json.dumps(r, indent=2)}"
print("✅ Video ready:", video_url)

#Step 6: Preview in Colab
HTML(f'<video src="{video_url}" controls autoplay loop width=720></video>')


<Response [200]>
Submit response: {
  "code": 0,
  "message": "SUCCEED",
  "request_id": "d0235a80-1ea8-42f6-90f5-3dd0aa70cb72",
  "data": {
    "task_id": "795961443710541919",
    "task_status": "submitted",
    "task_info": {},
    "created_at": 1757800515208,
    "updated_at": 1757800515208
  }
}
Job ID: 795961443710541919
[0] status: {
  "code": 0,
  "message": "SUCCEED",
  "request_id": "86062669-6d6f-42ac-b48e-9fb2a4e65369",
  "data": {
    "task_id": "795961443710541919",
    "task_status": "submitted",
    "task_info": {},
    "task_result": {},
    "task_status_msg": "",
    "created_at": 1757800515208,
    "updated_at": 1757800515208
  }
}
[1] status: {
  "code": 0,
  "message": "SUCCEED",
  "request_id": "91bbd679-8b69-47e7-abb0-0722f37e3262",
  "data": {
    "task_id": "795961443710541919",
    "task_status": "submitted",
    "task_info": {},
    "task_result": {},
    "task_status_msg": "",
    "created_at": 1757800515208,
    "updated_at": 1757800515208
  }
}
[2] status:

AssertionError: No video_url found! Final response:
{
  "code": 0,
  "message": "SUCCEED",
  "request_id": "0e9adf05-2818-4498-87e5-3c546b275fed",
  "data": {
    "task_id": "795961443710541919",
    "task_status": "succeed",
    "task_info": {},
    "task_result": {
      "videos": [
        {
          "id": "795961443777642501",
          "url": "https://v15-kling.klingai.com/bs2/upload-ylab-stunt-sgp/se/stream_lake_m2v_img2video_1080p_v16_distill/b6906618-e975-4022-8834-51fe38f6febc_raw_video.mp4?x-kcdn-pid=112372",
          "duration": "10.433"
        }
      ]
    },
    "task_status_msg": "",
    "created_at": 1757800515208,
    "updated_at": 1757800849718
  }
}